LSTM Model <br>
인공지능 모델 자체가 메모리를 가지고 있어서 이전 데이터를 가지고 후 데이터를 예측하는 모델


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime

Load Dataset from Yahoo Finance<br>
https://finance.yahoo.com/quote/005930.KS/history?p=005930.KS

In [22]:
CODEPATH = "D:/GitHub/stock_crypto_price_prediction_clone" # Modify 
DATAPATH = "D:/00_MLDL/00_BreadBrother/00_Stock" # Modify
data = pd.read_csv(f"{DATAPATH}/005930.KS.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-08-07,22400.0,22740.0,22300.0,22720.0,229.302444,13787100.0
1,2015-08-10,22800.0,22840.0,22600.0,22820.0,230.311722,5722500.0
2,2015-08-11,23020.0,23580.0,22960.0,23140.0,233.541336,11664300.0
3,2015-08-12,23000.0,23280.0,22800.0,23080.0,232.935745,11154000.0
4,2015-08-13,23060.0,23060.0,22760.0,22800.0,230.109848,7520550.0


Compute Mid Price

In [23]:
high_prices = data["High"].values
low_prices = data["Low"].values
mid_prices = (high_prices + low_prices) / 2

Create Windows

In [24]:
seq_len = 50
sequence_length = seq_len + 1

result = []

# 51개 데이터를 묶어서 리스트로 저장
# 1228-51 = 51 X 1177개의 array 생성
for index in range(len(mid_prices) - sequence_length) :
    result.append(mid_prices[index: index+sequence_length])

Normalize Data

In [32]:
normalize_data = []
for window in result:
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalize_data.append(normalized_window)

# 1177 X 51 array
normalized_result = np.array(normalize_data)

# Split train and test data
row = int(round(normalized_result.shape[0] * 0.9))
# 1059 X 51 array
train = normalized_result[:row, :]
np.random.shuffle(train)

# 1059 X 50
x_train = train[:, :-1]
# 1059 X 50 X 1
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# 1059 X 1
y_train = train[:, -1]

# 118 X 50
x_test = normalized_result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = normalized_result[row:, -1]

x_train.shape, x_test.shape

((1059, 50, 1), (118, 50, 1))

<h5>Build a Model</h5>

In [34]:
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


Training

In [36]:
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=10,
    epochs=20)

Train on 1059 samples, validate on 118 samples
Epoch 1/20


InternalError:  Blas GEMM launch failed : a.shape=(10, 50), b.shape=(50, 200), m=10, n=200, k=50
	 [[{{node lstm_3/while/body/_1/MatMul_1}}]] [Op:__inference_keras_scratch_graph_3114]

Function call stack:
keras_scratch_graph
